In [1]:
import sys
import os
sys.path.append('/eos/home/r/ruxie/.local/lib/python3.9/site-packages')

import math
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K

import plotly.graph_objects as go
import colorsys

from sklearn import metrics
# from dcor import distance_correlation as dcor 

# phi resolution factor for mapping φ to pixel bins
phi_res = 128 / (2 * math.pi) 
print(f"phi_res = {phi_res:.3f}")

# Print paths to verify
print(sys.path)

from qkeras import QActivation, QDense, quantized_relu, quantized_bits
from tensorflow.keras.models import Model
from tensorflow.keras.layers import PReLU, Input, Concatenate, Dense, ReLU, Dropout, BatchNormalization, Activation
from scipy.stats import pearsonr, linregress
from sklearn.impute import SimpleImputer
from tqdm import tqdm

from matplotlib import cm

import plotly.graph_objects as go
import colorsys

"""Fold batchnormalization with previous QDense layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import warnings

import numpy as np
import six


from qkeras.qlayers import QDense
from qkeras.quantizers import *

import tensorflow.compat.v2 as tf
from tensorflow.keras import layers
from tensorflow.python.framework import smart_cond as tf_utils
from tensorflow.python.ops import math_ops

tf.compat.v2.enable_v2_behavior()

class QDenseBatchnorm(QDense):
  """Implements a quantized Dense layer fused with Batchnorm."""

  def __init__(
    self,
    units,
    activation=None,
    use_bias=True,
    kernel_initializer="he_normal",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    kernel_quantizer=None,
    bias_quantizer=None,
    kernel_range=None,
    bias_range=None,

    # batchnorm params
    axis=-1,
    momentum=0.99,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    renorm=False,
    renorm_clipping=None,
    renorm_momentum=0.99,
    fused=None,
    trainable=True,
    virtual_batch_size=None,
    adjustment=None,

    # other params
    ema_freeze_delay=None,
    folding_mode="ema_stats_folding",
    **kwargs):

    super(QDenseBatchnorm, self).__init__(
      units=units,
      activation=activation,
      use_bias=use_bias,
      kernel_initializer=kernel_initializer,
      bias_initializer=bias_initializer,
      kernel_regularizer=kernel_regularizer,
      bias_regularizer=bias_regularizer,
      activity_regularizer=activity_regularizer,
      kernel_constraint=kernel_constraint,
      bias_constraint=bias_constraint,
      kernel_quantizer=kernel_quantizer,
      bias_quantizer=bias_quantizer,
      kernel_range=kernel_range,
      bias_range=bias_range,
      **kwargs)
    
    # initialization of batchnorm part of the composite layer
    self.batchnorm = layers.BatchNormalization(
      axis=axis, momentum=momentum, epsilon=epsilon, center=center,
      scale=scale, beta_initializer=beta_initializer,
      gamma_initializer=gamma_initializer,
      moving_mean_initializer=moving_mean_initializer,
      moving_variance_initializer=moving_variance_initializer,
      beta_regularizer=beta_regularizer,
      gamma_regularizer=gamma_regularizer,
      beta_constraint=beta_constraint, gamma_constraint=gamma_constraint,
      renorm=renorm, renorm_clipping=renorm_clipping,
      renorm_momentum=renorm_momentum, fused=fused, trainable=trainable,
      virtual_batch_size=virtual_batch_size, adjustment=adjustment
      )

    self.ema_freeze_delay = ema_freeze_delay
    assert folding_mode in ["ema_stats_folding", "batch_stats_folding"]
    self.folding_mode = folding_mode          

  def build(self, input_shape):
    super(QDenseBatchnorm, self).build(input_shape)

    # self._iteration (i.e., training_steps) is initialized with -1. When
    # loading ckpt, it can load the number of training steps that have been
    # previously trainied. If start training from scratch.
    # TODO(lishanok): develop a way to count iterations outside layer
    self._iteration = tf.Variable(-1, trainable=False, name="iteration",
                                  dtype=tf.int64)

  def call(self, inputs, training=None):

    # numpy value, mark the layer is in training
    training = self.batchnorm._get_training_value(training)  # pylint: disable=protected-access

    # checking if to update batchnorm params
    if (self.ema_freeze_delay is None) or (self.ema_freeze_delay < 0):
      # if ema_freeze_delay is None or a negative value, do not freeze bn stats
      bn_training = tf.cast(training, dtype=bool)
    else:
      bn_training = tf.math.logical_and(training, tf.math.less_equal(
        self._iteration, self.ema_freeze_delay))

    kernel = self.kernel
    
    #execute qdense output
    qdense_outputs = tf.keras.backend.dot(
      inputs, 
      kernel
      )

    if self.use_bias:
      bias = self.bias
      qdense_outputs = tf.keras.backend.bias_add(
        qdense_outputs, 
        bias,
        data_format="channels_last")
    else:
      bias = 0

    # begin batchnorm
    _ = self.batchnorm(qdense_outputs, training=bn_training)

    self._iteration.assign_add(tf_utils.smart_cond(
        training, lambda: tf.constant(1, tf.int64),
        lambda: tf.constant(0, tf.int64)))
    
    # calculate mean and variance from current batch
    bn_shape = qdense_outputs.shape
    ndims = len(bn_shape)
    reduction_axes = [i for i in range(ndims) if i not in self.batchnorm.axis]
    keep_dims = len(self.batchnorm.axis) > 1
    mean, variance = self.batchnorm._moments(  # pylint: disable=protected-access
        math_ops.cast(qdense_outputs, self.batchnorm._param_dtype),  # pylint: disable=protected-access
        reduction_axes,
        keep_dims=keep_dims)

    # get batchnorm weights
    gamma = self.batchnorm.gamma
    beta = self.batchnorm.beta
    moving_mean = self.batchnorm.moving_mean
    moving_variance = self.batchnorm.moving_variance

    if self.folding_mode == "batch_stats_folding":
      # using batch mean and variance in the initial training stage
      # after sufficient training, switch to moving mean and variance
      new_mean = tf_utils.smart_cond(
        bn_training, lambda: mean, lambda: moving_mean)
      new_variance = tf_utils.smart_cond(
        bn_training, lambda: variance, lambda: moving_variance)

      # get the inversion factor so that we replace division by multiplication
      inv = math_ops.rsqrt(new_variance + self.batchnorm.epsilon)
      if gamma is not None:
        inv *= gamma

      # fold bias with bn stats
      folded_bias = inv * (bias - new_mean) + beta

    elif self.folding_mode == "ema_stats_folding":
        # We always scale the weights with a correction factor to the long term
        # statistics prior to quantization. This ensures that there is no jitter
        # in the quantized weights due to batch to batch variation. During the
        # initial phase of training, we undo the scaling of the weights so that
        # outputs are identical to regular batch normalization. We also modify
        # the bias terms correspondingly. After sufficient training, switch from
        # using batch statistics to long term moving averages for batch
        # normalization.

        # use batch stats for calcuating bias before bn freeze, and use moving
        # stats after bn freeze
        mv_inv = math_ops.rsqrt(moving_variance + self.batchnorm.epsilon)
        batch_inv = math_ops.rsqrt(variance + self.batchnorm.epsilon)

        if gamma is not None:
            mv_inv *= gamma
            batch_inv *= gamma
        folded_bias = tf_utils.smart_cond(
          bn_training,
          lambda: batch_inv * (bias - mean) + beta,
          lambda: mv_inv * (bias - moving_mean) + beta)
        # moving stats is always used to fold kernel in tflite; before bn freeze
        # an additional correction factor will be applied to the conv2d output
        # end batchnorm 
        inv = mv_inv
    else:
        assert ValueError

    # wrap dense kernel with bn parameters
    folded_kernel = inv*kernel
    # quantize the folded kernel
    if self.kernel_quantizer is not None:
        q_folded_kernel = self.kernel_quantizer_internal(folded_kernel)
    else:
        q_folded_kernel = folded_kernel
    
    #quantize the folded bias
    if self.bias_quantizer_internal is not None:
        q_folded_bias = self.bias_quantizer_internal(folded_bias)
    else:
        q_folded_bias = folded_bias

    applied_kernel = q_folded_kernel
    applied_bias = q_folded_bias
    
    #calculate qdense output using the quantized folded kernel
    folded_outputs = tf.keras.backend.dot(inputs, applied_kernel)

    if training is True and self.folding_mode == "ema_stats_folding":
      batch_inv = math_ops.rsqrt(variance + self.batchnorm.epsilon)
      y_corr = tf_utils.smart_cond(
          bn_training,
          lambda: (math_ops.sqrt(moving_variance + self.batchnorm.epsilon) *
                   math_ops.rsqrt(variance + self.batchnorm.epsilon)),
          lambda: tf.constant(1.0, shape=moving_variance.shape))
      folded_outputs = math_ops.mul(folded_outputs, y_corr)

    folded_outputs = tf.keras.backend.bias_add(
      folded_outputs, 
      applied_bias,
      data_format="channels_last"
    )
    
    if self.activation is not None:
      return self.activation(folded_outputs)
    
    return folded_outputs

  def get_config(self):
    base_config = super().get_config()
    bn_config = self.batchnorm.get_config()
    config = {"ema_freeze_delay": self.ema_freeze_delay,
              "folding_mode": self.folding_mode}
    name = base_config["name"]
    out_config = dict(
        list(base_config.items())
        + list(bn_config.items()) + list(config.items()))

    # names from different config override each other; use the base layer name
    # as the this layer's config name
    out_config["name"] = name
    return out_config

  def get_quantization_config(self):
    return {
        "kernel_quantizer": str(self.kernel_quantizer_internal),
        "bias_quantizer": str(self.bias_quantizer_internal),
    }
    
  def get_quantizers(self):
    return self.quantizers

  # def get_prunable_weights(self):
  #   return [self.kernel]

  def get_folded_weights(self):
    """Function to get the batchnorm folded weights.
    This function converts the weights by folding batchnorm parameters into
    the weight of QDense. The high-level equation:
    W_fold = gamma * W / sqrt(variance + epsilon)
    bias_fold = gamma * (bias - moving_mean) / sqrt(variance + epsilon) + beta
    """

    kernel = self.kernel
    if self.use_bias:
      bias = self.bias
    else:
      bias = 0

    # get batchnorm weights and moving stats
    gamma = self.batchnorm.gamma
    beta = self.batchnorm.beta
    moving_mean = self.batchnorm.moving_mean
    moving_variance = self.batchnorm.moving_variance

    # get the inversion factor so that we replace division by multiplication
    inv = math_ops.rsqrt(moving_variance + self.batchnorm.epsilon)
    if gamma is not None:
      inv *= gamma

    # wrap conv kernel and bias with bn parameters
    folded_kernel = inv * kernel
    folded_bias = inv * (bias - moving_mean) + beta

    return [folded_kernel, folded_bias]

2025-09-18 06:40:03.479941: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-18 06:40:07.981248: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-18 06:40:07.985318: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-18 06:40:55.898470: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


phi_res = 20.372
['/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '', '/eos/user/r/ruxie/tf_env/lib64/python3.9/site-packages', '/eos/user/r/ruxie/tf_env/lib/python3.9/site-packages', '/eos/home/r/ruxie/.local/lib/python3.9/site-packages']


/eos/user/r/ruxie/tf_env/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [2]:
def load_and_process_anomalous_data(file_name):
    with h5py.File(file_name, 'r') as hf:
        nmuon, nLRjet, nSRjet, negamma, netau, njtau = 4, 6, 6, 4, 4, 4

        print(hf.keys())  # 可保留调试

        def fix_phi_range(phi_data):
            phi_fixed = phi_data % (2 * math.pi)
            phi_scaled = np.round(phi_fixed * 128 / (2 * math.pi))
            return phi_scaled % 128

        # 命名统一为 load_and_scale_both_versions
        def load_and_scale_both_versions(dataset, n_objects, scale_factor=10, eta_factor=40):
            raw = hf[dataset][:, 0:n_objects, :]

            scaled = np.copy(raw)
            scaled[:, :, 0] *= scale_factor
            scaled[:, :, 1] *= eta_factor
            scaled[:, :, 2] = fix_phi_range(scaled[:, :, 2])

            return (
                scaled.reshape(-1, 3 * n_objects),
                raw.reshape(-1, 3 * n_objects)
            )

        # 以下只加载 Topo 2A 所需部分
        L1_jFexSR_jets, L1_jFexSR_jets_raw = load_and_scale_both_versions('L1_jFexSR_jets', nSRjet)
        L1_eFex_taus, L1_eFex_taus_raw = load_and_scale_both_versions('L1_eFex_taus', netau)
        L1_muons, L1_muons_raw = load_and_scale_both_versions('L1_muons', nmuon, scale_factor=10000)

        # 处理 MET 数据（scaled 和 raw）
        L1_MET = hf['L1_MET'][:]
        L1_MET_raw = np.copy(L1_MET)

        L1_MET[:, 0] *= 10
        L1_MET[:, 2] = fix_phi_range(L1_MET[:, 2])
        L1_MET_scaled = np.stack([L1_MET[:, 0], L1_MET[:, 2]], axis=1)

        L1_MET_raw = np.stack([L1_MET_raw[:, 0], L1_MET_raw[:, 2]], axis=1)

        # 只保留必要字段
        pass_L1_unprescaled = hf["pass_L1_unprescaled"][:]

        # 合并 scaled 和 raw Topo 2A
        Topo_2A = np.concatenate([L1_jFexSR_jets, L1_eFex_taus, L1_muons, L1_MET_scaled], axis=1)
        Topo_2A_unscaled = np.concatenate([L1_jFexSR_jets_raw, L1_eFex_taus_raw, L1_muons_raw, L1_MET_raw], axis=1)

        # 与 background 对齐的 fill_median
        def fill_median(array):
            for i in range(array.shape[1]):
                col = array[:, i]
                median = np.nanmedian(col)
                array[np.isnan(col), i] = median
            return array

        Topo_2A = fill_median(Topo_2A)
        Topo_2A_unscaled = fill_median(Topo_2A_unscaled)

        return Topo_2A, Topo_2A_unscaled, pass_L1_unprescaled


In [3]:
Topo_2A_jz1, Topo_2A_jz1_unscaled, pass_L1_jz1 = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/MC_07-17-2024/jjJZ1_07-17-2024.h5')
Topo_2A_jz2_23e, Topo_2A_jz2_23e_unscaled, pass_L1_jz2_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_jjJZ2_04-15-2025.h5')
Topo_2A_jz4_23e, Topo_2A_jz4_23e_unscaled, pass_L1_jz4_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_jjJZ4_04-15-2025.h5')
Topo_2A_ttbar_1lep_23e, Topo_2A_ttbar_1lep_23e_unscaled, pass_L1_ttbar_1lep_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/misc/mc23e_ttbar_1lep.h5')
Topo_2A_ttbar_2lep_23e, Topo_2A_ttbar_2lep_23e_unscaled, pass_L1_ttbar_2lep_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_ttbar_2lep_04-15-2025.h5')
Topo_2A_ZZ4lep_23e, Topo_2A_ZZ4lep_23e_unscaled, pass_L1_ZZ4lep_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_ZZ4lep_04-15-2025.h5')

<KeysViewHDF5 ['HLT_MET', 'HLT_electrons', 'HLT_jets', 'HLT_muons', 'HLT_photons', 'L1_MET', 'L1_eFex_taus', 'L1_egammas', 'L1_jFexLR_jets', 'L1_jFexSR_jets', 'L1_jFex_taus', 'L1_muons', 'LRT_electrons', 'LRT_muons', 'pass_HLT_unprescaled', 'pass_L1_unprescaled']>
<KeysViewHDF5 ['EB_weights', 'HLT_MET_cell', 'HLT_MET_mhtpufit_pf', 'HLT_MET_mhtpufit_pf_subjesgscIS', 'HLT_MET_nn', 'HLT_MET_pfopufit', 'HLT_MET_tcpufit', 'HLT_electrons', 'HLT_jets', 'HLT_muons', 'HLT_photons', 'L1_MET', 'L1_eFex_taus', 'L1_egammas', 'L1_jFexLR_jets', 'L1_jFexSR_jets', 'L1_jFex_taus', 'L1_muons', 'LRT_electrons', 'LRT_muons', 'averageInteractionsPerCrossing', 'event_number', 'lumiBlock', 'lumiblock_bads', 'mu', 'pass_HLT_unprescaled', 'pass_L1_unprescaled', 'run_number']>
<KeysViewHDF5 ['EB_weights', 'HLT_MET_cell', 'HLT_MET_mhtpufit_pf', 'HLT_MET_mhtpufit_pf_subjesgscIS', 'HLT_MET_nn', 'HLT_MET_pfopufit', 'HLT_MET_tcpufit', 'HLT_electrons', 'HLT_jets', 'HLT_muons', 'HLT_photons', 'L1_MET', 'L1_eFex_taus', 

In [4]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
def Qmake_encoder_set_weights(input_dim,h_dim_1,h_dim_2,latent_dim):
    l2_factor = 1e-3
    inputs = keras.Input(shape=(input_dim), name = "inputs")
    x = QDenseBatchnorm(h_dim_1,
             kernel_quantizer=quantized_bits(10,5,0,alpha=1),
             bias_quantizer=quantized_bits(10,5,0,alpha=1),
             kernel_initializer=keras.initializers.HeNormal(seed=None),
             bias_initializer=keras.initializers.Zeros(),
            name = "dense1")(inputs)
    x = QActivation(quantized_relu(15, negative_slope=1/1024), name = 'relu1')(x)
    x = QDenseBatchnorm(h_dim_2,
             kernel_quantizer=quantized_bits(10,5,0,alpha=1),
             bias_quantizer=quantized_bits(10,5,0,alpha=1),
             kernel_initializer=keras.initializers.HeNormal(seed=None),
             bias_initializer=keras.initializers.Zeros(),
            name = "dense2")(x)
    x = QActivation(quantized_relu(15, negative_slope=1/1024), name = 'relu2')(x)
    z_mean=QDense(latent_dim, name='z_mean',
                  kernel_quantizer=quantized_bits(10,5,0,alpha=1),
                  bias_quantizer=quantized_bits(10,5,0,alpha=1),
                  kernel_initializer=keras.initializers.HeNormal(seed=None),
                  bias_initializer=keras.initializers.Zeros())(x)
    z_logvar=Dense(latent_dim, name='z_log_var',
                          kernel_initializer=keras.initializers.Zeros(),
                          bias_initializer=keras.initializers.Zeros())(x)
    z=Sampling()([z_mean,z_logvar])
    encoder = keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder
def Qmake_decoder_set_weights(input_dim,h_dim_1,h_dim_2,latent_dim):
    l2_factor = 1e-3
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2,
                   activation='relu',
                   kernel_initializer=keras.initializers.HeNormal(seed=None),
                   bias_initializer=keras.initializers.Zeros())(inputs)
    x = BatchNormalization(name="BN_decode1")(x)
    x=layers.Dense(h_dim_1,
                   activation='relu',
                   kernel_initializer=keras.initializers.HeNormal(seed=None),
                   bias_initializer=keras.initializers.Zeros())(x)
    x = BatchNormalization(name="BN_decode2")(x)
    z=layers.Dense(input_dim,
                   kernel_initializer=keras.initializers.HeNormal(seed=None),
                   bias_initializer=keras.initializers.Zeros())(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder
def Qmake_discriminator(input_dim, h_dim_1, h_dim_2):
    l2_factor = 1e-3
    inputs = keras.Input(shape=(input_dim))
    x = Dense(h_dim_1,
              activation='relu',
              kernel_initializer=keras.initializers.HeNormal(seed=None),
              bias_initializer=keras.initializers.Zeros())(inputs)
    x = Dense(h_dim_2,
              activation='relu',
              kernel_initializer=keras.initializers.HeNormal(seed=None),
              bias_initializer=keras.initializers.Zeros())(x)
    x = Dense(1,
              activation='sigmoid',  # Output probability
              kernel_initializer=keras.initializers.HeNormal(seed=None),
              bias_initializer=keras.initializers.Zeros())(x)
    discriminator = keras.Model(inputs, x, name='discriminator')
    return discriminator
def custom_mse_loss_with_multi_index_scaling(masked_data, masked_reconstruction):
    jet_scale = 1
    tau_scale = 1
    muon_sacle = 1
    met_scale = 1

    # Define the indices and their corresponding scale factors
    scale_dict = {
        0: jet_scale,
        3: jet_scale,
        6: jet_scale,
        9: jet_scale,
        12: jet_scale,
        15: jet_scale,
        18: tau_scale,
        21: tau_scale,
        24: tau_scale,
        27: tau_scale,
        30: muon_sacle,
        33: muon_sacle,
        36: muon_sacle,
        39: muon_sacle,
        42: met_scale
    }
    
    # Create the scaling tensor
    scale_tensor = tf.ones_like(masked_data)
    
    for index, factor in scale_dict.items():
        index_mask = tf.one_hot(index, depth=tf.shape(masked_data)[-1])
        scale_tensor += index_mask * (factor - 1)
    
    # Apply scaling
    scaled_data = masked_data * scale_tensor
    scaled_reconstruction = masked_reconstruction * scale_tensor
    
    ########
    # Hardcoded lists for eta and phi indices
    eta_indices = [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40]
    phi_indices = [2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 43]

    batch_size = tf.shape(scaled_reconstruction)[0]
    
    # Apply constraints to eta
    for i in eta_indices:
        indices = tf.stack([tf.range(batch_size), tf.fill([batch_size], i)], axis=1)
        updates = 3 * tf.tanh(scaled_reconstruction[:, i] / 3)
        scaled_reconstruction = tf.tensor_scatter_nd_update(scaled_reconstruction, indices, updates)
    
    # Apply constraints to phi
    for i in phi_indices:
        indices = tf.stack([tf.range(batch_size), tf.fill([batch_size], i)], axis=1)
        updates = 3.14159265258979*(10/8) * tf.tanh(scaled_reconstruction[:, i] / (3.14159265258979*(10/8)))
        scaled_reconstruction = tf.tensor_scatter_nd_update(scaled_reconstruction, indices, updates)
    #########

    # Calculate MSE using keras.losses.mse
    mse = keras.losses.mse(scaled_data, scaled_reconstruction)
    
    # Take the mean across all dimensions
    return tf.reduce_mean(mse)
class VAE_Model(keras.Model):
    def __init__(self, encoder, decoder, discriminator, steps_per_epoch=20, cycle_length=20, min_beta=0, max_beta=1, min_gamma=0, max_gamma=1, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder        
        self.discriminator = discriminator
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        
        self.discriminator_loss_tracker = keras.metrics.Mean(name="discriminator_loss")
        self.gamma_tracker = keras.metrics.Mean(name="gamma")
        
        self.beta_tracker = keras.metrics.Mean(name="beta")
        self.steps_per_epoch = steps_per_epoch
        self.cycle_length = tf.cast(cycle_length, tf.float32)
        self.min_beta = tf.cast(min_beta, tf.float32)
        self.max_beta = tf.cast(max_beta, tf.float32)
        self.beta = tf.Variable(min_beta, dtype=tf.float32)
        self.min_gamma = tf.cast(min_gamma, tf.float32)
        self.max_gamma = tf.cast(max_gamma, tf.float32)
        self.gamma = tf.Variable(min_gamma, dtype=tf.float32)

    def compile(self, optimizer, **kwargs):
        super(VAE_Model, self).compile(**kwargs)
        # Set the optimizer for the entire model (encoder + decoder + discriminator)
        self.optimizer = optimizer

        # Collect trainable variables from encoder, decoder, and discriminator
        trainable_variables = (
            self.encoder.trainable_weights + 
            self.decoder.trainable_weights + 
            self.discriminator.trainable_weights
        )
        # Build the optimizer with the full variable list
        self.optimizer.build(trainable_variables)

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
            self.discriminator_loss_tracker,
            self.beta_tracker,
        ]
    def cyclical_annealing_beta(self, epoch):
        cycle = tf.floor(1.0 + epoch / self.cycle_length)
        x = tf.abs(epoch / self.cycle_length - cycle + 1)
        # For first half (x < 0.5), scale 2x from 0 to 1
        # For second half (x >= 0.5), stay at 1
        scaled_x = tf.where(x < 0.5, 2.0 * x, 1.0)
        return self.min_beta + (self.max_beta - self.min_beta) * scaled_x
    def get_gamma_schedule(self, epoch):
        # Convert to float32 for TF operations
        epoch = tf.cast(epoch, tf.float32)
        
        # Calculate annealing progress
        anneal_progress = (epoch - 50.0) / 50.0
        gamma_anneal = self.min_gamma + (self.max_gamma - self.min_gamma) * anneal_progress
        
        # Implement the conditions using tf.where
        gamma = tf.where(epoch < 50.0, 
                        0.0,  # if epoch < 50
                        tf.where(epoch >= 100.0,
                                self.max_gamma,  # if epoch >= 100
                                gamma_anneal))   # if 50 <= epoch < 100
        
        return gamma
    
    def train_step(self, data):
        # Get the current epoch number
        epoch = tf.cast(self.optimizer.iterations / self.steps_per_epoch, tf.float32)
        
        # Update beta
        self.beta.assign(self.cyclical_annealing_beta(epoch))
        self.gamma.assign(self.get_gamma_schedule(epoch))

        # ---------------------------
        # Train the Discriminator
        # ---------------------------
        with tf.GradientTape() as tape_disc:
            # Generate reconstructed data
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            
            # Get discriminator predictions
            real_output = self.discriminator(data)
            fake_output = self.discriminator(reconstruction * mask)
            
            # Labels for real and fake data
            real_labels = tf.ones_like(real_output)
            fake_labels = tf.zeros_like(fake_output)
            
            # Discriminator loss
            d_loss_real = keras.losses.binary_crossentropy(real_labels, real_output)
            d_loss_fake = keras.losses.binary_crossentropy(fake_labels, fake_output)
            d_loss = d_loss_real + d_loss_fake
            d_loss = tf.reduce_mean(d_loss)
        
        grads_disc = tape_disc.gradient(d_loss, self.discriminator.trainable_weights)
        self.optimizer.apply_gradients(zip(grads_disc, self.discriminator.trainable_weights))

        # ---------------------------
        # Train the VAE (Generator)
        # ---------------------------
        with tf.GradientTape() as tape:
            data_noisy = data + tf.random.normal(tf.shape(data), 
                                               mean=0, 
                                               stddev=0)
            # Keep the mask logic
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            data_noisy = data_noisy * mask  # Apply mask to noisy data
            
            z_mean, z_log_var, z = self.encoder(data_noisy)
            reconstruction = self.decoder(z)
            reconstruction_loss = custom_mse_loss_with_multi_index_scaling(mask*data, mask*reconstruction)

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)

            # Generator (VAE) wants to fool the discriminator
            fake_output = self.discriminator(mask*reconstruction)
            valid_labels = tf.ones_like(fake_output)  # Try to make discriminator think reconstructions are real
            g_loss_adv = keras.losses.binary_crossentropy(valid_labels, fake_output)
            g_loss_adv = tf.reduce_mean(g_loss_adv)

            total_loss = reconstruction_loss + kl_loss * self.beta + g_loss_adv * self.gamma
            
        grads_vae = tape.gradient(total_loss, self.encoder.trainable_weights + self.decoder.trainable_weights)
        self.optimizer.apply_gradients(zip(grads_vae, self.encoder.trainable_weights + self.decoder.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.discriminator_loss_tracker.update_state(g_loss_adv)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "disc_loss": self.discriminator_loss_tracker.result(),
            "beta": self.beta,
            "raw_loss": self.reconstruction_loss_tracker.result() + self.kl_loss_tracker.result(),
            "w_kl_loss": self.kl_loss_tracker.result() * self.beta,
            "w_disc_loss": self.discriminator_loss_tracker.result() * self.gamma,
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = custom_mse_loss_with_multi_index_scaling(mask*data, mask*reconstruction)

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(kl_loss)
        # Discriminator loss (only for monitoring)
        # pass both data and reconstruction through D to get generator adversarial loss
        real_output = self.discriminator(data)
        fake_output = self.discriminator(mask*reconstruction)
        real_labels = tf.ones_like(real_output)
        fake_labels = tf.zeros_like(fake_output)
        d_loss_real = keras.losses.binary_crossentropy(real_labels, real_output)
        d_loss_fake = keras.losses.binary_crossentropy(fake_labels, fake_output)
        d_loss = d_loss_real + d_loss_fake
        d_loss = tf.reduce_mean(d_loss)
        
        # Generator adversarial loss
        valid_labels = tf.ones_like(fake_output)
        g_loss_adv = keras.losses.binary_crossentropy(valid_labels, fake_output)
        g_loss_adv = tf.reduce_mean(g_loss_adv)
        total_loss = reconstruction_loss + kl_loss * self.beta + g_loss_adv * self.gamma
        
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
            "raw_loss": reconstruction_loss + kl_loss,
            "disc_loss": g_loss_adv,
            "w_kl_loss": kl_loss * self.beta,
            "w_disc_loss": g_loss_adv * self.gamma
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

In [5]:
encoder = Qmake_encoder_set_weights(44, 32, 16, 3)
vae_dec = Qmake_decoder_set_weights(44, 32, 16, 3)
vae_disc = Qmake_discriminator(44, 8, 2)
vis_model = VAE_Model(encoder, vae_dec, vae_disc, steps_per_epoch=20, min_beta=0, max_beta=0.1, min_gamma=0.01, max_gamma=0.1)
vis_model.load_weights('/eos/user/r/ruxie/calibration/L1AD_sw_model/versionFDL-GAN_ALT/best_kl_loss/').expect_partial()
vis_encoder = vis_model.get_layer("encoder")

In [6]:
result = [7, 5, 5, 7, 5, 5, 5, 4, 4, 5, 4, 3, 4, 3, 3, 3, 3, 2, 6, 5, 5, 5, 4, 5, 4, 3, 4, 3, 2, 3, 4, 4, 4, 2, 3, 3, 1, 1, 2, -2, -1, -1, 6, 5]
def apply_power_of_2_scaling(X, result):
    # Apply the scaling using 2 raised to the power of the result
    X_scaled = X / (2.0 ** np.array(result))
    return X_scaled
Topo_2A_jz1 = apply_power_of_2_scaling(Topo_2A_jz1, result)
Topo_2A_jz2_23e = apply_power_of_2_scaling(Topo_2A_jz2_23e, result)
Topo_2A_jz4_23e = apply_power_of_2_scaling(Topo_2A_jz4_23e, result)
Topo_2A_ttbar_1lep_23e = apply_power_of_2_scaling(Topo_2A_ttbar_1lep_23e, result)
Topo_2A_ttbar_2lep_23e = apply_power_of_2_scaling(Topo_2A_ttbar_2lep_23e, result)
Topo_2A_ZZ4lep_23e = apply_power_of_2_scaling(Topo_2A_ZZ4lep_23e, result)

In [7]:
def compute_anomaly_score_exact(encoder, events):
    """
    Compute anomaly score exactly as in original code:
    mean(mu^2) per event, where mu = encoder output[0].
    """
    latent = encoder(events)       # returns [z_mean, z_logvar, z]
    mus = latent[0].numpy()        # mu = z_mean
    scores = np.mean(np.square(mus), axis=1)
    return scores
def find_anomalous_events_with_unscaled(signals_scaled, signals_unscaled, signal_names, encoder):
    results = {}
    for scaled, unscaled, name in zip(signals_scaled, signals_unscaled, signal_names):
        scores = compute_anomaly_score_exact(encoder, scaled)
        results[name] = {
            "scaled": scaled,
            "unscaled": unscaled,
            "scores": scores
        }
    return results

In [8]:
signals_scaled = [
    Topo_2A_jz1,
    Topo_2A_jz2_23e,
    Topo_2A_jz4_23e,
    Topo_2A_ttbar_1lep_23e,
    Topo_2A_ttbar_2lep_23e,
    Topo_2A_ZZ4lep_23e
]

signals_unscaled = [
    Topo_2A_jz1_unscaled,
    Topo_2A_jz2_23e_unscaled,
    Topo_2A_jz4_23e_unscaled,
    Topo_2A_ttbar_1lep_23e_unscaled,
    Topo_2A_ttbar_2lep_23e_unscaled,
    Topo_2A_ZZ4lep_23e_unscaled
]

signal_names = [
    'jz1',
    'jz2_23e',
    'jz4_23e',
    'ttbar_1lep_23e',
    'ttbar_2lep_23e',
    'ZZ4lep_23e'
]

events = find_anomalous_events_with_unscaled(signals_scaled, signals_unscaled, signal_names, encoder)

In [9]:
feature_names = []
for i in range(6):
    feature_names += [f'jFexSR_jet{i}_pt (GeV)', f'jFexSR_jet{i}_eta', f'jFexSR_jet{i}_phi (rad)']
for i in range(4):
    feature_names += [f'eFex_tau{i}_pt (GeV)', f'eFex_tau{i}_eta', f'eFex_tau{i}_phi (rad)']
for i in range(4):
    feature_names += [f'muon{i}_pt (GeV)', f'muon{i}_eta', f'muon{i}_phi (rad)']
feature_names += ['MET_pt (GeV)', 'MET_phi (rad)']

In [10]:
threshold_500Hz = 1521991 / (256 * 256 * 3)
threshold_1kHz  = 1333204 / (256 * 256 * 3)
threshold_2kHz = 1237997 / (256 * 256 * 3)
threshold_3kHz = 1156836 / (256 * 256 * 3)
threshold_4kHz = 1096728 / (256 * 256 * 3)
threshold_5kHz = 1051755 / (256 * 256 * 3)

In [12]:
from matplotlib.colors import LogNorm
def plot_anomaly_vs_kinematics(events, feature_names, save_dir):
    """
    events: dict, output of find_anomalous_events_with_unscaled
    feature_names: list of str
    save_dir: str, main folder to save plots
    """
    # different signals in different colors
    colors = itertools.cycle(plt.cm.tab10.colors) 
    
    for signal_name, color in zip(events.keys(), colors):
        data = events[signal_name]
        unscaled = data["unscaled"]
        scores = data["scores"]

        # a subfolder per process
        signal_dir = os.path.join(save_dir, signal_name) 
        os.makedirs(signal_dir, exist_ok=True)
        
        for i, feature in enumerate(feature_names):
            x = unscaled[:, i]
            y = scores

            plt.figure(figsize=(8, 6))  # increase fig size
            
            # Create 2D histogram with 50 bins in each dimension, using log scale for counts
            h = plt.hist2d(x, y, bins=50, cmap='viridis', cmin=1, norm=LogNorm())
            cbar = plt.colorbar(h[3], label='Log(Counts)')
            
            # exclude pt=0 for linear regression
            mask = x != 0
            if np.sum(mask) > 2:  # at least 3 data
                slope, intercept, r_value, p_value, std_err = linregress(x[mask], y[mask])
                x_fit = np.linspace(np.min(x[mask]), np.max(x[mask]), 100)
                y_fit = slope * x_fit + intercept
                plt.plot(x_fit, y_fit, color="red", linestyle="-", linewidth=1.5,
                         label=f"Linear fit (r={r_value:.3f})")
                x_fit_ext = np.linspace(0, np.min(x[mask]), 100)
                y_fit_ext = slope * x_fit_ext + intercept
                plt.plot(x_fit_ext, y_fit_ext, color="red", linestyle="--", linewidth=1.2)
            
            plt.xlabel(feature)
            plt.ylabel("Anomaly Score")
            plt.title(f"{feature} vs Anomaly Score ({signal_name})")

            threshold_lines = [
                (threshold_500Hz, 'red', '500Hz'),
                (threshold_1kHz, 'blue', '1kHz'),
                (threshold_2kHz, 'purple', '2kHz'),
                (threshold_3kHz, 'teal', '3kHz'),
                (threshold_4kHz, 'darkorange', '4kHz'),
                (threshold_5kHz, 'seagreen', '5kHz')
            ]
            
            for threshold, color_val, label in threshold_lines:
                plt.axhline(threshold, color=color_val, linestyle='--', linewidth=1.0, alpha=0.8)

            from matplotlib.lines import Line2D
            legend_elements = [
                Line2D([0], [0], color='red', linestyle='-', linewidth=1.5, label=f'Linear fit (r={r_value:.3f})' if np.sum(mask) > 2 else ''),
                *[Line2D([0], [0], color=color_val, linestyle='--', linewidth=1.0, label=label) 
                  for threshold, color_val, label in threshold_lines]
            ]
            
            legend_elements = [elem for elem in legend_elements if elem.get_label()]
            
            plt.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.98), 
                      framealpha=0.9, fontsize=9)
            
            plt.tight_layout()
            file_path = os.path.join(signal_dir, f"{feature}.png")
            plt.savefig(file_path, dpi=150, bbox_inches="tight")
            plt.close()

save_dir="0918_anomaly_vs_kinematics_legend_aside"
plot_anomaly_vs_kinematics(events, feature_names, save_dir)    

In [17]:
def plot_efficiency_vs_kinematics(events, feature_names, save_dir, bins):
    """
    Args:
        events: dict, output of find_anomalous_events_with_unscaled
        feature_names: list of str
        save_dir: str, main folder to save plots
        bins: int, number of bins
    """
    
    for process_name, data in events.items():
        unscaled = data["unscaled"]
        scores = data["scores"]
        
        process_dir = os.path.join(save_dir, process_name)
        os.makedirs(process_dir, exist_ok=True)
        
        for i, feature in enumerate(feature_names):
            values = unscaled[:, i]
            
            bin_edges = np.linspace(values.min(), values.max(), bins+1)
            bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
            
            pass_rate_500 = []
            pass_rate_1k  = []
            pass_rate_2k  = []
            pass_rate_3k  = []
            pass_rate_4k  = []
            pass_rate_5k  = []
            
            # record bins with events
            has_data_mask = []
            
            for j in range(bins):
                mask = (values >= bin_edges[j]) & (values < bin_edges[j+1])
                if np.any(mask):
                    pass_rate_500.append(np.mean(scores[mask] > threshold_500Hz))
                    pass_rate_1k.append(np.mean(scores[mask] > threshold_1kHz))
                    pass_rate_2k.append(np.mean(scores[mask] > threshold_2kHz))
                    pass_rate_3k.append(np.mean(scores[mask] > threshold_3kHz))
                    pass_rate_4k.append(np.mean(scores[mask] > threshold_4kHz))
                    pass_rate_5k.append(np.mean(scores[mask] > threshold_5kHz))
                    has_data_mask.append(True)
                else:
                    pass_rate_500.append(np.nan)
                    pass_rate_1k.append(np.nan)
                    pass_rate_2k.append(np.nan)
                    pass_rate_3k.append(np.nan)
                    pass_rate_4k.append(np.nan)
                    pass_rate_5k.append(np.nan)
                    has_data_mask.append(False)
            
            bin_centers = np.array(bin_centers)
            pass_rate_500 = np.array(pass_rate_500)
            pass_rate_1k = np.array(pass_rate_1k)
            pass_rate_2k = np.array(pass_rate_2k)
            pass_rate_3k = np.array(pass_rate_3k)
            pass_rate_4k = np.array(pass_rate_4k)
            pass_rate_5k = np.array(pass_rate_5k)
            has_data_mask = np.array(has_data_mask)
            
            plt.figure(figsize=(8,6))
            
            # only link dots representing bins with events
            def plot_with_data_mask(x, y, label, color, **kwargs):
                data_indices = np.where(has_data_mask)[0]
                if len(data_indices) == 0:
                    return
                
                segments = []
                current_segment = [data_indices[0]]
                for i in range(1, len(data_indices)):
                    if data_indices[i] == data_indices[i-1] + 1:
                        current_segment.append(data_indices[i])
                    else:
                        segments.append(current_segment)
                        current_segment = [data_indices[i]]
                segments.append(current_segment)
                
                # 为每个连续段画线（不绘制标记点）
                for segment in segments:
                    if len(segment) > 1:  # only link if at least 2 dots
                        plt.plot(x[segment], y[segment], color=color, label=label, **kwargs)
                    elif len(segment) == 1:  # 单个点也画线（长度为1的线）
                        plt.plot(x[segment], y[segment], color=color, label=label, **kwargs)
                
                # 移除重复的图例标签
                handles, labels = plt.gca().get_legend_handles_labels()
                by_label = dict(zip(labels, handles))
                plt.gca().legend(by_label.values(), by_label.keys())
            
            # 绘制各阈值曲线（去掉marker参数）
            plot_with_data_mask(bin_centers, pass_rate_500, label="500Hz", color="#f4ae6f", linewidth=1.5, alpha=0.8, linestyle='-')
            plot_with_data_mask(bin_centers, pass_rate_1k, label="1kHz", color="#99c9db", linewidth=1.5, alpha=0.8, linestyle='-')
            plot_with_data_mask(bin_centers, pass_rate_2k, label="2kHz", color="#c82423", linewidth=1.5, alpha=0.8, linestyle='-')
            plot_with_data_mask(bin_centers, pass_rate_3k, label="3kHz", color="#6668ae", linewidth=1.5, alpha=0.8, linestyle='-')
            plot_with_data_mask(bin_centers, pass_rate_4k, label="4kHz", color="#fa7f6f", linewidth=1.5, alpha=0.8, linestyle='-')
            plot_with_data_mask(bin_centers, pass_rate_5k, label="5kHz", color="#2878b4", linewidth=1.5, alpha=0.8, linestyle='-')
            
            plt.xlabel(feature)
            plt.ylabel("Trigger Efficiency")
            plt.ylim(0, 1.05)
            plt.title(f"Efficiency vs {feature} ({process_name})")
            
            plt.grid(True, alpha=0.3)
            
            file_path = os.path.join(process_dir, f"{feature}_Efficiency.png")
            plt.savefig(file_path, dpi=150, bbox_inches="tight")
            plt.close()

save_dir="0918_efficiency_vs_kinematics_10bins"
bins=10
plot_efficiency_vs_kinematics(events, feature_names, save_dir, bins)

In [15]:
# def heatmap_anomaly_vs_jetcount(events, save_dir, bins_score=50):
#     os.makedirs(save_dir, exist_ok=True)

#     jet_pt_indices = [i*3 for i in range(6)] 

#     for process_name, data in events.items():
#         unscaled = data["unscaled"]
#         scores = data["scores"]

#         jet_counts = np.sum(unscaled[:, jet_pt_indices] > 0.0, axis=1) # count jet number

#         x_bins = np.arange(jet_counts.min()-0.5, jet_counts.max()+1.5, 1)  # jet count is discrete
#         y_bins = np.linspace(scores.min(), scores.max(), bins_score)

#         hist, x_edges, y_edges = np.histogram2d(jet_counts, scores, bins=[x_bins, y_bins])

#         hist_log = np.log1p(hist.T)

#         fig, ax = plt.subplots(figsize=(6,5))
#         im = ax.imshow(hist_log, origin="lower", aspect="auto",
#                        extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]],
#                        cmap="viridis")
#         cbar = plt.colorbar(im, ax=ax)
#         cbar.set_label("log(Event Count + 1)")

#         nx, ny = hist_log.shape  # nx = score bins, ny = jet count bins after transpose
#         for ix in range(ny):
#             for iy in range(nx):
#                 val = hist_log[iy, ix]
#                 count = hist.T[iy, ix]
#                 if count > 0:
#                     x_text = (x_edges[ix] + x_edges[ix+1]) / 2
#                     y_text = (y_edges[iy] + y_edges[iy+1]) / 2
#                     ax.text(x_text, y_text, f"{val:.1f}",
#                             color="white" if val > hist_log.max()/2 else "black",
#                             ha="center", va="center", fontsize=7)

#         plt.axhline(threshold_500Hz, color='red', linestyle='--', label='Threshold 500Hz')
#         plt.axhline(threshold_1kHz, color='blue', linestyle='--', label='Threshold 1kHz')
#         plt.axhline(threshold_2kHz, color='purple', linestyle='--', label='Threshold 2kHz')
#         plt.axhline(threshold_3kHz, color='teal', linestyle='--', label='Threshold 3kHz')
#         plt.axhline(threshold_4kHz, color='darkorange', linestyle='--', label='Threshold 4kHz')
#         plt.axhline(threshold_5kHz, color='seagreen', linestyle='--', label='Threshold 5kHz')
#         ax.legend()

#         ax.set_xlabel(f"Jet Count (pt > 0)")
#         ax.set_ylabel("Anomaly Score")
#         ax.set_title(f"Anomaly Score vs Jet Count ({process_name})")

#         plt.savefig(os.path.join(save_dir, f"{process_name}.png"), dpi=300)
#         plt.close()
# save_dir="0827_anomaly_vs_jetcount"
# heatmap_anomaly_vs_jetcount(events, save_dir, bins_score=50)

In [16]:
# def heatmap_anomaly_vs_bijet_invariant_mass(events, save_dir, bins_score=25):
#     os.makedirs(save_dir, exist_ok=True)

#     jet0_idx = [0, 1, 2]
#     jet1_idx = [3, 4, 5]

#     def calc_mjj(jet0, jet1):
#         pt0, eta0, phi0 = jet0
#         pt1, eta1, phi1 = jet1

#         # E = pt*cosh(eta), px=pt*cos(phi), py=pt*sin(phi), pz=pt*sinh(eta)
#         E0 = pt0 * np.cosh(eta0)
#         px0 = pt0 * np.cos(phi0)
#         py0 = pt0 * np.sin(phi0)
#         pz0 = pt0 * np.sinh(eta0)

#         E1 = pt1 * np.cosh(eta1)
#         px1 = pt1 * np.cos(phi1)
#         py1 = pt1 * np.sin(phi1)
#         pz1 = pt1 * np.sinh(eta1)

#         E_tot = E0 + E1
#         px_tot = px0 + px1
#         py_tot = py0 + py1
#         pz_tot = pz0 + pz1

#         mjj2 = E_tot**2 - px_tot**2 - py_tot**2 - pz_tot**2
#         mjj2 = np.maximum(mjj2, 0)
#         return np.sqrt(mjj2)

#     for process_name, data in events.items():
#         unscaled = data["unscaled"]
#         scores = data["scores"]

#         # 提取 leading jets 并计算 mjj
#         jet0 = unscaled[:, jet0_idx]
#         jet1 = unscaled[:, jet1_idx]
#         mjj = np.array([calc_mjj(j0, j1) for j0, j1 in zip(jet0, jet1)])

#         # 定义 bin
#         x_bins = np.linspace(mjj.min(), mjj.max(), 50)
#         y_bins = np.linspace(scores.min(), scores.max(), bins_score)

#         # 2D 直方图
#         hist, x_edges, y_edges = np.histogram2d(mjj, scores, bins=[x_bins, y_bins])

#         hist_log = np.log1p(hist.T)

#         fig, ax = plt.subplots(figsize=(6,5))
#         im = ax.imshow(hist_log, origin="lower", aspect="auto",
#                        extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]],
#                        cmap="viridis")
#         cbar = plt.colorbar(im, ax=ax)
#         cbar.set_label("log(Event Count + 1)")

#         # 显示每个 bin 的数值
#         # nx, ny = hist_log.shape
#         # for ix in range(ny):
#         #     for iy in range(nx):
#         #         val = hist_log[iy, ix]
#         #         count = hist.T[iy, ix]
#         #         if count > 0:
#         #             x_text = (x_edges[ix] + x_edges[ix+1]) / 2
#         #             y_text = (y_edges[iy] + y_edges[iy+1]) / 2
#         #             ax.text(x_text, y_text, f"{val:.1f}",
#         #                     color="white" if val > hist_log.max()/2 else "black",
#         #                     ha="center", va="center", fontsize=7)

#         plt.axhline(threshold_500Hz, color='red', linestyle='--', label='Threshold 500Hz')
#         plt.axhline(threshold_1kHz, color='blue', linestyle='--', label='Threshold 1kHz')
#         plt.axhline(threshold_2kHz, color='purple', linestyle='--', label='Threshold 2kHz')
#         plt.axhline(threshold_3kHz, color='teal', linestyle='--', label='Threshold 3kHz')
#         plt.axhline(threshold_4kHz, color='darkorange', linestyle='--', label='Threshold 4kHz')
#         plt.axhline(threshold_5kHz, color='seagreen', linestyle='--', label='Threshold 5kHz')
#         ax.legend()

#         ax.set_xlabel("Invariant Mass of Leading Jets (GeV)")
#         ax.set_ylabel("Anomaly Score")
#         ax.set_title(f"Anomaly Score vs Leading Jets Mjj ({process_name})")

#         plt.savefig(os.path.join(save_dir, f"{process_name}.png"), dpi=300)
#         plt.close()
# save_dir="0830_anomaly_vs_bijet_mass"
# heatmap_anomaly_vs_bijet_invariant_mass(events, save_dir, bins_score=25)

In [17]:
# def plot_anomaly_vs_muoncount_heatmap(events, save_dir, pt_threshold=0.0, bins_score=50):
#     os.makedirs(save_dir, exist_ok=True)

#     # 每个 muon 的 pt 在 unscaled 中的位置 (4个 muons, 每个3个特征: pt, eta, phi)
#     muon_pt_indices = [18 + i*3 for i in range(4)]  

#     for process_name, data in events.items():
#         unscaled = data["unscaled"]
#         scores = data["scores"]

#         # 计算 muon 数量
#         muon_counts = np.sum(unscaled[:, muon_pt_indices] > pt_threshold, axis=1)

#         # 定义 bin
#         x_bins = np.arange(muon_counts.min()-0.5, muon_counts.max()+1.5, 1)  # muon count 离散
#         y_bins = np.linspace(scores.min(), scores.max(), bins_score)

#         # 2D 直方图
#         hist, x_edges, y_edges = np.histogram2d(muon_counts, scores, bins=[x_bins, y_bins])

#         # 转置用于 imshow
#         hist_log = np.log1p(hist.T)

#         fig, ax = plt.subplots(figsize=(6,5))
#         im = ax.imshow(hist_log, origin="lower", aspect="auto",
#                        extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]],
#                        cmap="viridis")
#         cbar = plt.colorbar(im, ax=ax)
#         cbar.set_label("log(Event Count + 1)")

#         # 显示每个 bin 的数值
#         nx, ny = hist_log.shape
#         for ix in range(ny):
#             for iy in range(nx):
#                 val = hist_log[iy, ix]
#                 count = hist.T[iy, ix]
#                 if count > 0:
#                     x_text = (x_edges[ix] + x_edges[ix+1]) / 2
#                     y_text = (y_edges[iy] + y_edges[iy+1]) / 2
#                     ax.text(x_text, y_text, f"{val:.1f}",
#                             color="white" if val > hist_log.max()/2 else "black",
#                             ha="center", va="center", fontsize=7)

#         # 画阈值水平线
#         ax.axhline(threshold_500Hz, color='red', linestyle='--', label='500Hz threshold')
#         ax.axhline(threshold_1kHz, color='orange', linestyle='--', label='1kHz threshold')
#         ax.legend()

#         ax.set_xlabel(f"Muon Count (pt > {pt_threshold:.1f})")
#         ax.set_ylabel("Anomaly Score")
#         ax.set_title(f"Anomaly Score vs Muon Count ({process_name})")

#         plt.savefig(os.path.join(save_dir, f"{process_name}.png"), dpi=300)
#         plt.close()
# save_dir="anomaly_vs_muoncount_heatmap"
# plot_anomaly_vs_muoncount_heatmap(events, save_dir, pt_threshold=0.0, bins_score=50)